<h1>Applied Data Science Capstone Week3</h1>

<h3>All three parts are within one file</h3>
<h3>Part 1</h3>

In [1]:
# importing libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

In [2]:
# getting data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_data= requests.get(url).text
#parse HTML codes
soup_data = bs(raw_data, 'html.parser')
#print(soup_data.prettify())
header = soup_data.find_all("table")[0].find("tr")
print(header)

<tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>


In [3]:
data = []
list_header = []
for items in header:
    try:
        sub_item = items.get_text()
        list_header.append(sub_item.rstrip())
    except:
        continue
HTML_data = soup_data.find_all("table")[0].find_all("tr")[1:]
#print(HTML_data)
for element in HTML_data:
    sub_data = []
    for sub_element in element:
        try:
            sub_sub_element = sub_element.get_text()
            sub_data.append(sub_sub_element.rstrip())
        except:
            continue
    data.append(sub_data)
#convert to dataframe
df = pd.DataFrame(list(data), columns = list_header)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#Ignore cells with a borough that is Not assigned.
#If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
df = df[df.Borough != 'Not assigned']
not_assigned_nei = df['Neighbourhood'] == 'Not assigned'
df['Neighbourhood'][not_assigned_nei] = df['Borough'][not_assigned_nei]
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
#Merge rows that with same postal code
df.to_dict('series')
foo = lambda a: ", ".join(a) 
df2 = df.groupby(by = 'Postal Code').agg({'Borough':'first','Neighbourhood':foo}).reset_index()
df2.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df2.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [6]:
# show the rows that already merged
print(df.loc[df['Postal Code'] == 'M5A'])

  Postal Code           Borough              Neighbourhood
4         M5A  Downtown Toronto  Regent Park, Harbourfront


In [7]:
df2.shape

(103, 3)

<h3>Part2</h3>

In [8]:
#!pip install geocoder
import geocoder # import geocoder

In [9]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
#df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_geo.rename(columns ={"Postal Code":"PostalCode"}, inplace = True)
df_geo

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [10]:
df3 = df2.merge(df_geo)
df3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


<h3>Part 3</h3>

In [11]:
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [12]:
CLIENT_ID = 'ZATDKOURVOB4AJ3IIXSUAGCRRHDXZEA3EQLVZ4JZKLTAC53F' # your Foursquare ID
CLIENT_SECRET = 'I23YRMSDJN2POURVLYUA4FQ3YPS5MB33ZJACN4BGSZLFRFMC' # your Foursquare Secret
ACCESS_TOKEN = '2FENZ0FVTWEG2MSPQJD5X15JOMEIH1XG321FZ4OKA05HKKIO' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZATDKOURVOB4AJ3IIXSUAGCRRHDXZEA3EQLVZ4JZKLTAC53F
CLIENT_SECRET:I23YRMSDJN2POURVLYUA4FQ3YPS5MB33ZJACN4BGSZLFRFMC


In [13]:
address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


<h4>search Chinese food within 500 metres</h4>

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for borough, neighborhood, lat, lng in zip(df3['Borough'], df3['Neighborhood'],df3['Latitude'], df3['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
Toronto_data = df3[df3['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:

# create map of xxx toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for label, lat, lng in zip(Toronto_data['Neighborhood'], Toronto_data['Latitude'], Toronto_data['Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:

#function to get nearby venues from json into pandas.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
         # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [18]:
LIMIT = 50
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [19]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton, Parkdale Village, Exhibition Place",26,26,26,26,26,26
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,50,50,50,50,50,50
Christie,16,16,16,16,16,16
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
Davisville,35,35,35,35,35,35


In [20]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 213 uniques categories.


In [21]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
Toronto_onehot.shape

(1178, 213)

In [23]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.038462,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.020000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.020000
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.040000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.020000,0.02,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [24]:
Toronto_grouped.shape

(39, 213)

In [25]:
num_top_venues = 3

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.06
2   Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1          Bakery  0.08
2  Breakfast Spot  0.08


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.13
1      Farmers Market  0.07
2       Auto Workshop  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.18
1  Sandwich Place  0.06
2            Café  0.06


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12


----Church and Wellesley----
                 venue  freq
0          Coffee

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Cheese Shop,Bakery,Restaurant,Breakfast Spot,Beach
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Yoga Studio,Bar,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Park,Comic Shop,Skate Park,Brewery,Spa,Restaurant,Burrito Place,Farmers Market,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Airport Gate,Rental Car Location,Boat or Ferry,Sculpture Garden
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Office,Juice Bar,Middle Eastern Restaurant,Miscellaneous Shop
5,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Nightclub
6,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Gay Bar,Yoga Studio,Men's Store,Café,Ice Cream Shop,Salon / Barbershop
7,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,Gym,Japanese Restaurant,Deli / Bodega,Beer Bar,Seafood Restaurant,American Restaurant
8,Davisville,Dessert Shop,Sandwich Place,Gym,Pizza Place,Café,Sushi Restaurant,Italian Restaurant,Coffee Shop,Discount Store,Seafood Restaurant
9,Davisville North,Food & Drink Shop,Park,Breakfast Spot,Sandwich Place,Gym / Fitness Center,Department Store,Dance Studio,Hotel,Wine Bar,Dog Run


In [28]:

from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[:])

[1 1 1 1 0 0 1 0 0 0 1 0 2 0 1 1 0 1 3 1 2 0 1 0 0 0 2 4 0 1 1 1 1 0 0 1 0
 0 1]


In [30]:
Toronto_merged = Toronto_data

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Pub,Health Food Store,Wine Bar,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Pub
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Sandwich Place,Fast Food Restaurant,Park,Board Shop,Light Rail Station,Brewery,Liquor Store,Italian Restaurant,Restaurant,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Gastropub,Bakery,Brewery,Café,American Restaurant,Yoga Studio,Convenience Store,Cheese Shop,Pharmacy
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Swim School,Park,Business Service,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [33]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for nei, cluster, lat, lon in zip(Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels'], Toronto_merged['Latitude'], Toronto_merged['Longitude']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters